# S.VI.C: ORB-SLAM3

This notebook plots and analyzes the results shown in Figure 7g -- 7n of Section VI.C: "ORB-SLAM3."

Reproducing the results shown in the paper requires an installation of ROS and ORB-SLAM3 (modified to support elastic adjustment of task periods -- available at https://github.com/a01ixxx/orb-slam-elastic). The installation process is complex and requires several dependencies, some of which interfere with the FIMS requirements.

Therefore, to simplify reproducability of the artifact, we are providing direct access via TeamViewer to our evaluation workstation running Ubuntu 16 with LITMUS^RT. Connection instructions are provided in the artifact document.

We have also provided all relevant data traces here, and this notebook reproduces all results in Section VI.C: "ORB-SLAM3" of the paper from the raw data.

## Profiling Execution Times

We first profile the execution times of each ORB-SLAM3 pipeline task, using calls to `clock_gettime()` with the `CLOCK_THREAD_CPUTIME_ID.`

The following code reproduces the results shown in Figure 7j -- 7m.

### Obtain Execution Times

We have already provided the results data in the files under `orbslam3_results/execution_time_data`.

This can be reproduced on the evaluation platform by opening 3 terminal windows.

**Terminal 1:**
* Run: `roscore`
* Wait for message "started core service [/rosout]"

**Terminal 2:**
* Run:
* `cd ~/orbslam3`
* `rosrun ORB_SLAM3 Stereo_Inertial Vocabulary/ORBvoc.txt /home/ao/orbslam3/Examples_old/Stereo-Inertial/EuRoC.yaml true`
* Wait about 10-15 seconds for message "Hello Whale"

**Terminal 3:**
* Run `rosbag play ~/Downloads/MH_01_easy.bag /cam0/image_raw:=/camera/left/image_raw /cam1/image_raw:=/camera/right/image_raw /imu0:=/imu`
* Wait about 3 minutes for completion

Once complete, copy the output files `*_exe_times_file.txt` to `orbslam3_results/execution_time_data` in this repository.

### Plot and Analyze Results

The following code plots Figure Figure 7j -- 7m, showing execution time distributions. It also reports the worst observed execution times for each task.

In [ ]:
# Import Packages
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Define Style for Plotting

# Use the fivethirtyeight style
plt.style.use('fivethirtyeight')

# Define a palette
colors = sns.color_palette("deep", 5)

# Functions for plotting

# paths to read
paths = [] 
prefix = "execution_time_data/"
# prefix = "../data/orb_execution_times/histo_data/MH03/experiment_1/"
paths.append('us_imu_exe_times_file')
paths.append('us_left_camera_exe_times_file')
paths.append('ms_tracking_exe_times_file')
paths.append('ms_ba_exe_times_file')

# units
units = []
units.append('us')
units.append('us')
units.append('ms')
units.append('ms')

def plot_execution_times(i) :
    
    fig, ax = plt.subplots(figsize=(4.6, 2.5))
    data = []
    # read data
    with open(prefix + paths[i] + '.txt', 'r') as file:
        for line in file:
            content = line.split(',')
            data.append (float(content[1]))

    # Create histogram
    ax.hist(data, bins=30, density=True, alpha=0.8, edgecolor='black', linewidth=1.5, color=colors[i])
    ax.set_yscale('log')
    ax.set_xlabel('Execution Time (' + units[i] + ')', )
    ax.set_ylabel('Frequency')

    # Display the plot
    plt.tight_layout()

    # Save the figure
    fig.savefig(paths[i] + '.eps', format="eps")
    fig.savefig(paths[i] + '.png', format="png")
    fig.show()

    return np.max(data)

### Figure 7j

Plots the execution times of the IMU task and prints the worst-observed execution time $C_i$

In [ ]:
max_imu = plot_execution_times(0)
print(f"Maximum observed execution time of IMU task: {max_imu/1000} ms")

### Figure 7k

Plots the execution times of the Camera workload

In [ ]:
max_cam = plot_execution_times(1)
print(f"Maximum observed execution time of Camera task: {max_cam/1000} ms")

### Figure 7l

Plots the execution times of the Tracking workload and prints the worst-observed execution time $C_i$ of the combined Camera + Tracking task.

In [ ]:
max_track = plot_execution_times(2)
max_cam_track = max_cam/1000+max_track
print(f"Maximum observed execution time of Tracking task: {max_track} ms")
print(f"Maximum observed execution time of Camera + Tracking task: {max_cam_track} ms")

### Figure 7m

Plots the execution times of the Mapping task and prints the worst-observed execution time $C_i$

In [ ]:
max_map = plot_execution_times(3)
print(f"Maximum observed execution time of Mapping task: {max_map} ms")

## Assigning Elasticity Values

We nesxt assign elasticity values by measuring the loss associated with adjusting task periods individually. In our case, loss is defined as 10000X, where X is the relative translational error (RTE) of the completed map in units of meters from the ground truth.

Obtained traces are stored in `orbslam3_results/elasticity_data` and corresponding ground truth data are stored in `orbslam3_results/groundtruth`.

The following functions are used to calculate RTE.

Helper functions:

In [ ]:
# Import Packages
import statistics
import copy
from pyquaternion import Quaternion
from math import *

def relative_translation(p1, p2):
    return [p1[0] - p2[0], p1[1] - p2[1], p1[2] - p2[2]]


def rotation_error(r1, r2):
    diff_r = Quaternion(r1.elements - r2.elements)
    return diff_r.degrees

def translation_error(p1, p2):
    return sqrt(pow(p1[0] - p2[0],2) + pow(p1[1] - p2[1],2) + pow(p1[2] - p2[2],2))

def traj_translation_error(trj1, gt_traj):
    gt_index = 0
    errors = []
    for p in trj1:
        while gt_index < len(gt_traj) and p.timestamp > gt_traj[gt_index].timestamp:
            gt_index = gt_index + 1
        
        if gt_index < len(gt_traj):
            errors.append(translation_error(p.position, gt_traj[gt_index].position))

    return errors

def traj_relative_translation_error(trj1, gt_traj):
    gt_index = 0
    gt_last_index = 0


    timestamps = []
    errors = []

    last_p = None

    for p in trj1:
        while gt_index < len(gt_traj) and p.timestamp >= gt_traj[gt_index].timestamp:
            gt_index = gt_index + 1
        
        if gt_index < len(gt_traj):
            if last_p != None:
                relative_trans = relative_translation(p.position,last_p.position)
                relative_trans_gt = relative_translation(gt_traj[gt_index].position, gt_traj[gt_last_index].position)

                timestamps.append(p.timestamp)
                errors.append(translation_error(relative_trans, relative_trans_gt))

            last_p = p
            gt_last_index = gt_index

    return timestamps, errors

def traj_relative_orientation_error(trj1, gt_traj):
    gt_index = 0
    gt_last_index = 0
    errors = []

    last_p = None

    for p in trj1:
        while gt_index < len(gt_traj) and p.timestamp >= gt_traj[gt_index].timestamp:
            gt_index = gt_index + 1
        
        if gt_index < len(gt_traj):
            if last_p != None:

                e = rotation_error(p.orientation, gt_traj[gt_index].orientation)

            last_p = p
            gt_last_index = gt_index

    return errors

def translation(p, t):
    return np.array([p[0]-t[0], p[1]-t[1], p[2]-t[2]])

def distance(p1, p2):
    return sqrt(pow(p1[0] - p2[0], 2) + pow(p1[1] - p2[1], 2))

def error_log(errors):
    print('Mean : ', statistics.mean(errors), '    std dev  :  ',  statistics.stdev(errors), '    Max  :  ',  max(errors), '    Min  :  ',  min(errors))


class loc_point:
    def __init__(self):
        self.timestamp = 0
        self.position = None
        self.orientation = None

def rte(gt_traj, trace_traj) :
    baseline_timestamps, baseline_rpe_error = traj_relative_translation_error(trace_traj, gt_traj)
    return sum(baseline_rpe_error)/len(baseline_rpe_error)

The following function loads ground truth data.

In [ ]:
def load_gt(gt_path) :
    
    offset_flag = True
    offset_p = [] #np.array()
    offset_r = Quaternion()
    offset_point = None
    conj_offset_r = Quaternion()
    ground_truth_traj = []

    start_time = 0

    gtxs = []
    gtys = []
    gtzs = []

    with open(gt_path) as f:
        lines = f.readlines()
        for line in lines:

            # This is the first line
            if line.split(',')[0] == '#timestamp':
                continue

            content = [float(x) for x in line.split(',')]

            new_p = loc_point()

            new_p.timestamp = content[0] / (1.0 * 10e8)
            if start_time == 0:
                start_time = content[0] / (1.0 * 10e8)

            new_p.position = np.array([content[1], content[2], content[3]])
            new_p.orientation = Quaternion(content[4], content[5], content[6], content[7])

            if offset_flag:
                offset_point = copy.copy(new_p)
                conj_offset_r = offset_point.orientation.conjugate
                offset_flag = False
            
            #### Eliminate the initial offset
            new_p.position = translation(new_p.position, offset_point.position)
            new_p.position = conj_offset_r.rotate(new_p.position)

            new_p.orientation = Quaternion(new_p.orientation.elements - offset_point.orientation.elements)

            ### coordination transform
            new_p.position = np.array([new_p.position[1], -new_p.position[0], new_p.position[2]])

            ground_truth_traj.append(new_p)

            gtxs.append(new_p.position[0])
            gtys.append(new_p.position[1])
            gtzs.append(new_p.position[2]) 

    return ground_truth_traj, start_time

The following function loads experimental trace data.

In [ ]:
def load_trace(trace_name) :
        
    trace_xs = []
    trace_ys = []
    trace_zs = []
    trace_traj = []
    trace_timestamps = []

    with open(trace_name) as f:
        lines = f.readlines()
        for line in lines:
            content = [float(x) for x in line.split()]

            new_p = loc_point()
            new_p.timestamp = content[0]
            trace_timestamps.append(content[0])

            new_p.position = np.array([content[1], content[2], content[3]])
            new_p.orientation = Quaternion(content[4], content[5], content[6], content[7])

            trace_traj.append(new_p)

            trace_xs.append(new_p.position[0])
            trace_ys.append(new_p.position[1])
            trace_zs.append(new_p.position[2])

    return trace_traj

In [ ]:
# Load ground truth
gt_traj, start_time = load_gt("groudtruth/mh01.csv")

The following function shows the error for each task period, fits as a function of square deviation in rate, and plots against the error values. It also reports the derived elasticity according to Equation 23.

In [ ]:
def fit_weight(periods, errs, num=0) :
    R = 1 / (periods / 1000)
    R_max = R[0]
    X = (R_max - R) ** 2
    Y = errs * 10000

    if num == 0 :
        num = X.size

    slope, intercept = np.polyfit(X[0:num], Y[0:num], 1)


    # To visualize
    plt.figure(figsize=(4.6, 2.5))
    plt.scatter(X, Y, color='blue')
    plt.plot(X, slope * X + intercept, color='red', label='Best Fit')
    plt.xlabel(r'$(R^{max} - R)^2$') 
    plt.ylabel('Error')
    plt.tight_layout()
    plt.legend()
    plt.show()

    return slope

def compute_elasticity(c, w) :
    e = c ** 2 / w
    print(f'E: {e}')


### Compute Elasticity for IMU Task

We compute the errors obtained by adjusting IMU periods from 5--20ms, then plot and fit to compute the elasticity according to Equation 23 in the paper.

#### Compute Errors

In [ ]:
rte_imu = np.zeros(4)
trace_traj = load_trace("elasticity_data/imu_1/FrameTrajectory_TUM_Format.txt")
rte_imu[0] = rte(gt_traj, trace_traj)
trace_traj = load_trace("elasticity_data/imu_2/FrameTrajectory_TUM_Format.txt")
rte_imu[1] = rte(gt_traj, trace_traj)
trace_traj = load_trace("elasticity_data/imu_3/FrameTrajectory_TUM_Format.txt")
rte_imu[2] = rte(gt_traj, trace_traj)
trace_traj = load_trace("elasticity_data/imu_4/FrameTrajectory_TUM_Format.txt")
rte_imu[3] = rte(gt_traj, trace_traj)
print(rte_imu)

#### Figure 7g

The following code plots Figure 7g, fitting errors for each IMU period from 5--20.

In [ ]:
periods = np.arange(5,21,5)
w_imu = fit_weight(periods,rte_imu)
compute_elasticity(max_imu/1000, w_imu)


### Compute Elasticity for Mapping Task

We compute the errors obtained by adjusting mapping periods from 50--200ms, then plot and fit to compute the elasticity according to Equation 23 in the paper.

#### Compute Errors

In [ ]:
# TODO: Obtain correct data
rta_ba = np.zeros(4)
trace_traj = load_trace("elasticity_data/ba_1_new/FrameTrajectory_TUM_Format.txt")

#### Figure 7i

The following code plots Figure 7i, fitting errors for each mapping period from 50--200.

In [ ]:
#TODO: Clean up
periods = np.arange(50,201,50)
errs = np.array([0.00189339,0.002014436,0.002182768,0.002338362])
w_map = fit_weight(periods,errs)

w_map = 0.15807
# max_map = 285

compute_elasticity(max_map, w_map)

### Compute Elasticity for Camera + Tracking Task

We compute the errors obtained by adjusting camera and tracking periods from 50--250ms, then plot and fit to compute the elasticity according to Equation 23 in the paper.

#### Compute Errors

In [ ]:
rte_cam = np.zeros(5)
trace_traj = load_trace("elasticity_data/img_1/FrameTrajectory_TUM_Format.txt")
rte_cam[0] = rte(gt_traj, trace_traj)
trace_traj = load_trace("elasticity_data/image_2/FrameTrajectory_TUM_Format.txt")
rte_cam[1] = rte(gt_traj, trace_traj)
trace_traj = load_trace("elasticity_data/image_3/FrameTrajectory_TUM_Format.txt")
rte_cam[2] = rte(gt_traj, trace_traj)
trace_traj = load_trace("elasticity_data/image_4/FrameTrajectory_TUM_Format.txt")
rte_cam[3] = rte(gt_traj, trace_traj)
trace_traj = load_trace("elasticity_data/image_5/FrameTrajectory_TUM_Format.txt")
rte_cam[4] = rte(gt_traj, trace_traj)
print(rte_cam)

#### Figure 7h

The following code plots Figure 7h. We plotg errors for each camera and tracking period from 50--250, but only fit from 50--200, due to the instability at 250ms.

Note also that when increasing the image and tracking periods, the mapping period is also increased, so we adjust the first-order weight to ignore its impact.

In [ ]:
periods = np.arange(50,251,50)
w_cam = fit_weight(periods,rte_cam, 4)
w_cam = w_cam - w_map # Remove first-order effect of mapping
compute_elasticity(max_cam_track, w_cam)

## Evaluation of Online Adjustment

We artificially limit the CPU utilization available to ORB-SLAM3 by pinning all of its threads on a single core and restrict its total bandwidth with Linux cgroups. We compare the accuracy achieved by an unmodified run of ORB-SLAM3 to our elastic model, and to the adaptive approach presented in:

> A. Li, H. Liu, J. Wang, and N. Zhang, “From timing variations to performance degradation: Understanding and mitigating the impact of software execution timing in slam,” in IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS), 2022.

Corresponding traces are stored in:
* online_adjustment_results/baseline/FrameTrajectory_TUM_Format.txt
* online_adjustment_results/iros/FrameTrajectory_TUM_Format.txt
* online_adjustment_results/elastic_model/FrameTrajectory_TUM_Format.txt
* online_adjustment_results/tuned_elastic_model/FrameTrajectory_TUM_Format.txt

### Figure 7n

The following code loads the traces, compares them to the ground-truth data, then plots the RTE in meters, reproducing Figure 7n.


In [ ]:
# Load traces

baseline_traj = load_trace("online_adjustment_results/baseline/FrameTrajectory_TUM_Format.txt")
iros_traj = load_trace("online_adjustment_results/iros/FrameTrajectory_TUM_Format.txt")
elastic_traj = load_trace("online_adjustment_results/elastic_model/FrameTrajectory_TUM_Format.txt")
tuned_elastic_traj = load_trace("online_adjustment_results/tuned_elastic_model/FrameTrajectory_TUM_Format.txt")

# Downsample for better visualization

baseline_traj = baseline_traj[::10]
iros_traj = iros_traj[::10]
elastic_traj = elastic_traj[::10]
tuned_elastic_traj = tuned_elastic_traj[::10]

# Get errors and timestamps

baseline_timestamps, baseline_rpe_error = traj_relative_translation_error(baseline_traj, gt_traj)
iros_timestamps, iros_rpe_error = traj_relative_translation_error(iros_traj, gt_traj)
elastic_timestamps, elastic_rpe_error = traj_relative_translation_error(elastic_traj, gt_traj)
tuned_elastic_timestamps, tuned_elastic_rpe_error = traj_relative_translation_error(tuned_elastic_traj, gt_traj)

baseline_timestamps = [x - start_time for x in baseline_timestamps]
iros_timestamps = [x - start_time for x in iros_timestamps]
elastic_timestamps = [x - start_time for x in elastic_timestamps]
tuned_elastic_timestamps = [x - start_time for x in tuned_elastic_timestamps]

# Plot

fig, ax = plt.subplots(figsize=(9, 4.5))

ax.plot(baseline_timestamps, baseline_rpe_error, label='Baseline')
ax.plot(iros_timestamps, iros_rpe_error, label='Li et al.')
ax.plot(elastic_timestamps, elastic_rpe_error, label='Elastic model')
ax.plot(tuned_elastic_timestamps, tuned_elastic_rpe_error, label= 'Tuned elastic model')

ax.legend(fontsize=16)
ax.set_xlabel('Time [S]', fontsize=16)
ax.set_ylabel('RTE [m] (log-scale)', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.tick_params(axis='both', which='minor', labelsize=16)

ax.set_yscale('log')

plt.tight_layout()

plt.savefig('max_iterations.eps', format='eps')
plt.savefig('max_iterations.png', format='png')
plt.show()

### Statistics

We additionally report a few additional statistics.

In [ ]:
print("Time baseline: ", baseline_traj[-1].timestamp - baseline_traj[0].timestamp)
print("Time IROS: ",  iros_traj[-1].timestamp - iros_traj[0].timestamp)
print("Time Elastic: ",  elastic_traj[-1].timestamp - elastic_traj[0].timestamp)
print("Time Tuned Elastic: ", tuned_elastic_traj[-1].timestamp - tuned_elastic_traj[0].timestamp)

print("ATE baseline: ", sum(baseline_rpe_error)/len(baseline_rpe_error))
print("ATE IROS: ", sum(iros_rpe_error)/len(iros_rpe_error))
print("ATE Elastic: ", sum(elastic_rpe_error)/len(elastic_rpe_error))
print("ATE Tuned Elastic: ", sum(tuned_elastic_rpe_error)/len(tuned_elastic_rpe_error))